# AlexNet

## Imports

In [1]:
import os
import torch
import torchvision
from alexnet_model import AlexNet
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../helper_functions')

from train import train_model
from eval import set_seed, compute_confusion_matrix
from plot import plot_training_loss, plot_accuracy, plot_confusion_matrix
from dataset_loader import get_dataloaders_cifar10, unNormalize

## Initialize Setting

In [2]:
RANDOM_SEED = 123
BATCH_SIZE  = 256
NUM_EPCOHS  = 200

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device: ", DEVICE)
set_seed(RANDOM_SEED)

Device:  cuda:0


## Applying Transforms to the Dataset

In [3]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),
    torchvision.transforms.CenterCrop((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transforms  = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),
    torchvision.transforms.CenterCrop((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    num_workers=2
)

for images, labels in train_loader:
    print('Image batch dimensions: ', images.shape)
    print('Image label dimensions: ', labels.shape)
    print('Class labels of 10 examples: ', labels[:10])
    break

Files already downloaded and verified
Image batch dimensions:  torch.Size([256, 3, 64, 64])
Image label dimensions:  torch.Size([256])
Class labels of 10 examples:  tensor([4, 7, 4, 6, 2, 6, 9, 7, 3, 0])


## Training

In [4]:
model = AlexNet(num_classes=10)
model = model.to(DEVICE)
print("Device: ", DEVICE)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

mini_batch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPCOHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE,
    logging_interval=100,
    scheduler=scheduler,
    scheduler_on='valid_acc'
)

Device:  cuda:0
Epoch: 001/200 | Batch 0000/0175 | Loss: 2.3021
Epoch: 001/200 | Batch 0100/0175 | Loss: 2.1945
Epoch: 001/200 | Train: 19.49% | Validation: 19.92%
Time elapsed: 0.47 min
Epoch: 002/200 | Batch 0000/0175 | Loss: 1.9668


ConnectionResetError: [Errno 104] Connection reset by peer

BrokenPipeError: [Errno 32] Broken pipe

KeyboardInterrupt: 

In [ ]:
os.makedirs('save_model', exist_ok=True)
torch.save(model.state_dict(), 'save_model/AlexNet.pt')

## Plots

In [ ]:
plot_training_loss(mini_batch_loss_list=mini_batch_loss_list,
                   num_epoch=NUM_EPCOHS,
                   iter_per_epoch=len(train_loader),
                   result_dir=None,
                   averaging_iteration=200
                   )

plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None
)
plt.ylim([10, 100])
plt.show()

model.cpu()
class_dict = {0: 'airplane',
              1: 'automobile',
              2: 'bird',
              3: 'cat',
              4: 'deer',
              5: 'dog',
              6: 'frog',
              7: 'horse',
              8: 'ship',
              9: 'truck'}

mat = compute_confusion_matrix(model=model, data_loader=test_loader, device=torch.device('cpu'))
plot_confusion_matrix(mat, figsize=(10, 10), show_absolute=True, class_names=class_dict.values())
plt.show()